In [ ]:
%matplotlib inline

In [ ]:
import os
import re
import sys
import glob
import time
import logging
import tarfile
import datetime
import numpy as np
import urllib.request
import matplotlib
import radarkit
import chart
import data
import openpyxl

if not sys.version_info.major is 3:
    print('PyRadarKit requires Python 3')
    raise

In [ ]:
logger = logging.getLogger('iRadar')
logfile = os.path.expanduser('~/Documents/iRadar/logs/summary-{}.log'.format(time.strftime('%Y%m%d', time.localtime(time.time()))))
logging.basicConfig(filename=logfile, level=logging.INFO, format='%(asctime)s %(message)s', datefmt='%Y/%m/%d %H:%M:%S')
logging.Formatter.converter = time.gmtime
logger.setLevel(logging.INFO)

# level = logging.INFO

# if len(logger.handlers) == 0:
#     handler = logging.StreamHandler()
#     handler.setLevel(level)
#     handler.setFormatter(logging.Formatter('%(asctime)s %(message)s', datefmt='%Y/%m/%d %H:%M:%S'))
#     logger.addHandler(handler)

In [ ]:
def extract(archive, path='.'):
    zfile = None
    files = []
    tar = tarfile.open(archive)
    for info in tar.getmembers():
        logger.debug(info.name)
        basename = os.path.basename(info.name)
        if os.path.splitext(basename)[-1] == '.nc':
            tar.extract(info, path=path)
            files.append('{}/{}'.format(path, basename))
            symbols = re.findall(r'(?<=[0-9]-)[A-Za-z]+(?=.nc)', basename)
            if len(symbols) == 0:
                logger.info(archive)
                logger.info(info)
                logger.info(basename)
                continue
            symbol = symbols[0]
            if symbol == 'Z':
                zfile = '{}/{}'.format(path, basename)
    tar.close()
    return zfile, files

def timestrFromFilename(filename):
    t = re.findall(r'(?<=[A-Z]-)20[0-9][0-9][01][0-9][0-3][0-9]-[0-2][0-9][0-5][0-9][0-5][0-9](?=-E.*)', filename)[0]
    #return time.strftime('%Y/%m/%d %H:%M:%S', time.strptime(t, '%Y%m%d-%H%M%S'))
    #return time.strptime(t, '%Y%m%d-%H%M%S')
    return datetime.datetime.strptime(t, '%Y%m%d-%H%M%S')

In [ ]:
folders = sorted(glob.glob('/Volumes/Data/PX1000/20[0-9][0-9][01][0-9][0-3][0-9]'))

In [ ]:
folders

In [ ]:
def inspectFolder(folder):
    # List all files in the folder
    files = glob.glob('{}/_original/*'.format(folder))

    # First things first, get the start and end times of operation
    time_start = timestrFromFilename(files[0])
    time_end = timestrFromFilename(files[-1])
    count = len(files)
    logger.info(folder)
    logger.info('Start time = {}   End time = {}   Count = {}'.format(time_start, time_end, count))

    t1 = 0
    dic = {}
    scan_time = np.zeros(count)
    elevation = np.zeros(count)
    for k, file in enumerate(files):
        # Time
        f = re.findall(r'(?<=[A-Z]-)20[0-9][0-9][01][0-9][0-3][0-9]-[0-2][0-9][0-5][0-9][0-5][0-9](?=-[AEN].*)', file)
        t = f[0]
        t0 = time.mktime(time.strptime(t, '%Y%m%d-%H%M%S'))
        if t1 == 0 or t0 - t1 > 60:
            period = 60
        else:
            period = t0 - t1
        scan_time[k] = period
        t1 = t0
        # Elevation
        f = re.findall(r'(?<=-E)[0-9\.]+(?=.t[a-z\.]+)', file)
        if len(f) == 0:
            # Could be RHI, spotlight, etc.
            continue
        e = f[0]
        elevation[k] = e
        if e not in dic:
            dic[e] = 1
        else:
            dic[e] += 1
    # Pick an elevation
    ee = []
    for key in dic:
        ee.append(float(key))
    for elev in np.sort(ee):
        n = dic[str(elev)]
        logger.debug('E{} -> {:.4f}'.format(elev, n / count))
        if n / count > 0.1:
            break
    stride = max(1, int(count / 50))
    duration = scan_time.sum()

    logger.info('Total scan time = {} s'.format(duration))
    logger.info('Selected elevation = {}   stride = {}'.format(elev, stride))
    
    # Subset data files from the selected elevation
    patt = '{}/_original/[A-Z]*E{:.1f}.*'.format(folder, elev)
    files = glob.glob(patt)
    
    # Use the Z file to intelligently gather all product files through PyRadarKit
    base = 0
    green = 0
    orange = 0
    maroon = 0
    total = 0
    for file in files[::stride]:
        # If the supplied file is an archive, extract the files into a temp folder
        if os.path.splitext(file)[-1] == '.xz':
            tmp_folder = '/mnt/ramdisk'
            if not os.path.exists(tmp_folder):
                tmp_folder = '/Volumes/RAMDisk'
            if not os.path.exists(tmp_folder):
                tmp_folder = 'tmp'           
            if not os.path.exists(tmp_folder):
                os.mkdir(tmp_folder)
            logger.debug('Extracting {} ...'.format(file))
            zfile, extracted = extract(file, path=tmp_folder)
            if zfile is None:
                logger('Archive {} has no Z file'.format(file))
                continue
            logger.debug('Zfile = {}'.format(zfile))
        if not os.path.exists(zfile):
            logger.warning('File {} does not exist'.format(file))
            continue
        sweep = radarkit.read(zfile)
        # Remove the previously extracted files, if exist
        for tmp in extracted:
            os.remove(tmp) 
        # Get the reflectivity data to assess the shades
        if 'Z' not in sweep['products']:
            logger.info(file)
            logger.info(zfile)
            logger.info(sweep['products'].keys())
            continue
        data = np.nan_to_num(sweep['products']['Z']['data'])
        base += np.sum(data > 5)
        green += np.sum(np.logical_and(data >= 20, data < 35))
        orange += np.sum(np.logical_and(data >= 35, data < 50))
        maroon += np.sum(data >= 50)
        total += data.size
    logger.info('Base = {} / {} -> {:.2f} %   Green = {:.2f} %   Orange = {:.2f} %   Maroon = {:.2f} %'.format(
        base, total, 100.0 * base / total, 100.0 * green / base, 100.0 * orange / base, 100 * maroon / base))

    # Text description based on base, green, orange, maroon, etc.
    comment = []
    if base / total < 0.05:
        comment.append('Clear Air')
    if orange / base >= 0.1:
        comment.append('Heavy rain')
    elif green / base > 0.15:
        comment.append('Rain')
    if maroon / base >= 0.01:
        comment.append('Severe storms')
    comment = ', '.join(comment)
    logger.info('Comment = {}'.format(comment))
    
    return time_start, time_end, count, duration, comment


def addEntry(folder, row=None):
    time_start, time_end, count, duration, comment = inspectFolder(folder)
    
    # Spreadsheet
    filename = '/Users/boonleng/Downloads/datalog.xlsx'
    book = openpyxl.load_workbook(filename)
    sheet = book[book.sheetnames[0]]
    if row is None:
        row = sheet.max_row + 1
    
    # Start Time
    cell = sheet.cell(row=row, column=1)
    cell.value = time_start
    # cell.style = style_date
    cell.number_format = 'YYYY/MM/DD HH:MM'

    # End Time
    cell = sheet.cell(row=row, column=2)
    cell.value = time_end
    cell.number_format = 'YYYY/MM/DD HH:MM'

    # Count
    cell = sheet.cell(row=row, column=3)
    cell.value = count

    # Duration
    cell = sheet.cell(row=row, column=4)
    cell.value = '=TEXT(B{}-A{}, "h:mm")'.format(row, row)
    cell.alignment = openpyxl.styles.Alignment(horizontal='right')

    # Volume Time (ratio of a day)
    cell = sheet.cell(row=row, column=5)
    cell.value = min(0.9999, duration / 86400)
    cell.number_format = 'h:mm'

    # Comment
    cell = sheet.cell(row=row, column=6)
    cell.value = comment
    
    book.save(filename)
    book.close()

In [ ]:
for k, folder in enumerate(folders):
    addEntry(folder, row=k+551)

In [ ]:
inspectFolder(folders[0])